From C++ to Python
====

### Function declaration

In [1]:
%%file add.hpp
#ifndef ADD_HPP
#define ADD_HPP
int add(int a, int b);
#endif

Overwriting add.hpp


### Function definition

In [2]:
%%file add.cpp
int add(int a, int b) {
    return a+b;
}

Overwriting add.cpp


### Driver program for testing in C++

In [3]:
%%file driver.cpp
#include <iostream>
#include "add.hpp"
using std::cout;
using std::endl;

int main() {
    cout << add(3, 4) << endl;
}

Overwriting driver.cpp


### Compile and execute

In [4]:
%%bash
g++ -o driver.exe driver.cpp add.cpp
./driver.exe

7


## Using Cython to build a shared library

### Helper file to expose C++ functions to Python

In [5]:
%%file plus.pyx

cdef extern from 'add.cpp':
    int add(int a, int b)
    
def plus(a, b):
    return add(a, b)

Overwriting plus.pyx


### Method 1: Quick and dirty testing

For single session use, you can use the `pyximprot` module to compile the C/C++ library and import the pyx file as though it was a regular Python module.

In [6]:
import pyximport
pyximport.install()

import plus

plus.plus(3, 4)

7

### Method 2: Using distuils

The standard approach is to write a `setup.py` file that will build the shared library. The build moudle can then be used elsewhere or installed togetehr with your Python package.

In [7]:
%%file setup.py 
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize(
           "plus.pyx",   
            sources=["add.cpp"], 
            language="c++",        
      ))

Overwriting setup.py


### Build module

In [8]:
%%bash
python setup.py build_ext --inplace

Compiling plus.pyx because it changed.
Cythonizing plus.pyx
running build_ext
building 'plus' extension
gcc -pthread -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/opt/conda/include/python3.4m -c plus.cpp -o build/temp.linux-x86_64-3.4/plus.o
g++ -pthread -shared -L/opt/conda/lib -Wl,-rpath=/opt/conda/lib,--no-as-needed build/temp.linux-x86_64-3.4/plus.o -L/opt/conda/lib -lpython3.4m -o /home/jovyan/work/sta-663-2016/lectures/plus.cpython-34m.so


cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++ [enabled by default]
/opt/conda/lib/python3.4/site-packages/Cython/Compiler/Main.py:514: UserWarning: got unknown compilation option, please remove: sources
  warnings.warn(message)


### Using C++ function in Pyhton

In [9]:
import plus

plus.plus(3, 4)

7